In [1]:
import re
import json
import argparse
import os
import sys
import torch
import pandas as pd
import numpy as np

from tqdm import tqdm
from nltk.stem import PorterStemmer
# from transformers import GPT2Tokenizer, GPT2Model
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

from swisscom_ai.research_keyphrase.preprocessing.postagging import PosTaggingCoreNLP
from swisscom_ai.research_keyphrase.model.input_representation import InputTextObj
from swisscom_ai.research_keyphrase.model.extractor import extract_candidates

c:\Users\user0\anaconda3\envs\LLM_ENV\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
host = 'localhost'
port = 9005
pos_tagger = PosTaggingCoreNLP(host, port)

# load stopwords
stopwords = []
with open('UGIR_stopwords.txt', "r") as f:
    for line in f:
        if line:
            stopwords.append(line.replace('\n', ''))

stemmer = PorterStemmer()

def read_jsonl(path):
    data = []
    with open(path, 'r') as f:
        for line in f:
            item = json.loads(line.strip())
            data.append(item)
    return data

In [3]:
def get_candidates(core_nlp, text):
    tagged = core_nlp.pos_tag_raw_text(text)
    text_obj = InputTextObj(tagged, 'en')
    candidates = extract_candidates(text_obj)
    return candidates

def get_phrase_indices(text_tokens, phrase, prefix):
    text_tokens = [t.replace(prefix, '') for t in text_tokens]

    phrase = phrase.replace(' ', '')

    matched_indices = []
    matched_index = []
    target = phrase
    for i in range(len(text_tokens)):
        cur_token = text_tokens[i]
        sub_len = min(len(cur_token), len(phrase))
        if cur_token[:sub_len].lower() == target[:sub_len]:
            matched_index.append(i)
            target = target[sub_len:]
            if len(target) == 0:
                matched_indices.append([matched_index[0], matched_index[-1] + 1])
                target = phrase
        else:
            matched_index = []
            target = phrase
            if cur_token[:sub_len].lower() == target[:sub_len]:
                matched_index.append(i)
                target = target[sub_len:]
                if len(target) == 0:
                    matched_indices.append([matched_index[0], matched_index[-1] + 1])
                    target = phrase

    return matched_indices

def remove_repeated_sub_word(candidates_pos_dict):
    for phrase in candidates_pos_dict.keys():
        split_phrase = re.split(r'\s+|-', phrase)
        split_phrase = list(filter(None, split_phrase))
        if len(split_phrase) > 1:
            for word in split_phrase:
                if word in candidates_pos_dict:
                    single_word_positions = candidates_pos_dict[word]
                    phrase_positions = candidates_pos_dict[phrase]
                    single_word_alone_positions = [pos for pos in single_word_positions if not any(
                        pos[0] >= phrase_pos[0] and pos[1] <= phrase_pos[1] for phrase_pos in phrase_positions)]
                    candidates_pos_dict[word] = single_word_alone_positions

    return candidates_pos_dict

def get_all_indices(candidates_pos_dict, window_end):
    all_indices = []
    for phrase in candidates_pos_dict.values():
        for element in phrase:
            start_index = element[0]
            end_index = element[1]
            if(start_index>window_end):
                all_indices.extend(range(start_index, end_index))
    all_indices = sorted(all_indices)
    all_indices = list(dict.fromkeys(all_indices))

    return all_indices

def aggregate_phrase_scores(index_list, tokens_scores):
    total_score = 0.0

    for p_index in index_list:
        part_sum = tokens_scores[p_index[0]:p_index[1]].sum()
        total_score += part_sum

    return total_score

def get_score_full(candidates, references, maxDepth=15):
    precision = []
    recall = []
    reference_set = set(references)
    referencelen = len(reference_set)
    true_positive = 0
    for i in range(maxDepth):
        if len(candidates) > i:
            kp_pred = candidates[i]
            if kp_pred in reference_set:
                true_positive += 1
            precision.append(true_positive / float(i + 1))
            recall.append(true_positive / float(referencelen))
        else:
            precision.append(true_positive / float(len(candidates)))
            recall.append(true_positive / float(referencelen))
    return precision, recall


def evaluate_document(candidates, ground_truth):
    results = {}
    precision_scores, recall_scores, f1_scores = {5: [], 10: [], 15: []}, \
                                                 {5: [], 10: [], 15: []}, \
                                                 {5: [], 10: [], 15: []}
    for candidate, gt in zip(candidates, ground_truth):
        p, r = get_score_full(candidate, gt)
        for i in [5, 10, 15]:
            precision = p[i - 1]
            recall = r[i - 1]
            if precision + recall > 0:
                f1_scores[i].append((2 * (precision * recall)) / (precision + recall))
            else:
                f1_scores[i].append(0)
            precision_scores[i].append(precision)
            recall_scores[i].append(recall)

    print("########################\nMetrics")
    for i in precision_scores:
        print("@{}".format(i))
        print("F1:{}".format(np.mean(f1_scores[i])))
        print("P:{}".format(np.mean(precision_scores[i])))
        print("R:{}".format(np.mean(recall_scores[i])))

        top_n_p = 'precision@' + str(i)
        top_n_r = 'recall@' + str(i)
        top_n_f1 = 'f1@' + str(i)
        results[top_n_p] = np.mean(precision_scores[i])
        results[top_n_r] = np.mean(recall_scores[i])
        results[top_n_f1] = np.mean(f1_scores[i])
    print("#########################")

    return results

def evaluate_dataset(predicted_top, dataset, score_type, dataset_name):
    experiment_results = []
    gt_keyphrase_list = []
    predicted_keyphrase_list = []

    for i in range(len(dataset)):
        predicted_keyphrase = predicted_top[i]
        predicted_keyphrase = [phrase.lower() for phrase in predicted_keyphrase]
        predicted_keyphrase_list.append(predicted_keyphrase)

        stemmed_gt_keyphrases = [" ".join(stemmer.stem(word) for word in phrase.split()) for phrase in dataset[i]['keywords'].split(";")]
        gt_keyphrase = [key.lower() for key in stemmed_gt_keyphrases]
        gt_keyphrase_f = list(dict.fromkeys(gt_keyphrase))
        gt_keyphrase_list.append(gt_keyphrase_f)

    total_score = evaluate_document(predicted_keyphrase_list, gt_keyphrase_list)
    experiment_results.append(total_score)

    df = pd.DataFrame(experiment_results)

    path = f'experiment_results/{dataset_name}/'
    os.makedirs(path, exist_ok=True)
    df.to_csv(f'{path}score_type_{score_type}.csv', index=False)

    top3_f1_5 = df.nlargest(3, 'f1@5').reset_index(drop=True)
    top3_f1_10 = df.nlargest(3, 'f1@10').reset_index(drop=True)
    top3_f1_15 = df.nlargest(3, 'f1@15').reset_index(drop=True)

    return top3_f1_5, top3_f1_10, top3_f1_15

def get_same_len_segments(total_tokens_ids, max_len):
    num_of_seg = (len(total_tokens_ids) // max_len) + 1
    seg_len = int(len(total_tokens_ids) / num_of_seg)
    segments = []
    attn_masks = []
    for _ in range(num_of_seg):
        if len(total_tokens_ids) > seg_len:
            segment = total_tokens_ids[:seg_len]
            total_tokens_ids = total_tokens_ids[seg_len:]
        else:
            segment = total_tokens_ids
        segments.append(segment)
        attn_masks.append([1] * len(segments[-1]))

    return segments, attn_masks


def clean_data(text,min_length=10):
    pattern_long_numbers = re.compile(r'\b[0-9.,\s]{'+str(min_length)+r',}\b')
    aaa_pattern = r'(AAAA\s*)+'

    text = re.sub(aaa_pattern, '', text)
    text = re.sub(r' -LSB-\s*\d+\s*-RSB-', '', text)
    text = re.sub(r' -LSB-\s*(\d+\s*,\s*){1,7}\d+\s*-RSB-', '', text)
    text = re.sub(r'-LRB- ', '(', text)
    text = re.sub(r' -RRB-', ')', text)
    cleaned_text = re.sub(pattern_long_numbers, '', text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()
    
    return cleaned_text

<img src="hugging_token.png" width=1000px></img>

In [4]:
login(token="YOUR_TOKEN_HERE")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to C:\Users\user0\.cache\huggingface\token
Login successful


In [5]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3-8B", attn_implementation="eager", output_attentions=True)
prefix = 'Ġ'

c:\Users\user0\anaconda3\envs\LLM_ENV\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  2.83s/it]
c:\Users\user0\anaconda3\envs\LLM_ENV\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"device: {device}")

device: cuda


<h2>DATASET: SEMEVAL 2010</h2>

In [7]:
dataset_name = "semeval_test"
dataset = read_jsonl("KEYWORD_DATA/{}.json".format(dataset_name))

NON-PARAMETRIC SAMRANK GLOBAL SCORE

In [ ]:
model.to(device)
model.eval()

max_len = 1024

dataset_att_scores_overall = []

for data in tqdm(dataset):
    with torch.no_grad():
        input_text = data["title"] + ". " + data["abstract"] + " " + clean_data(data["fulltext"])
        tokenized_content = tokenizer(input_text, return_tensors='pt')

        candidates = get_candidates(pos_tagger, input_text)
        candidates = [phrase for phrase in candidates if phrase.split(' ')[0] not in stopwords]

        total_tokens_ids = tokenized_content['input_ids'].squeeze(0).tolist()
        total_tokens_ids = total_tokens_ids[1:]

        windows, attention_masks = get_same_len_segments(total_tokens_ids, max_len)

        phrase_score_dict = {}

        for i in range(len(windows)):
            window = windows[i]
            attention_mask = attention_masks[i]
            window = [128000] + window
            attention_mask = [1] + attention_mask
            window = torch.tensor([window])
            attention_mask = torch.tensor([attention_mask])

            outputs = model(window.to(device), attention_mask=attention_mask.to(device))
            attentions = outputs.attentions
            del outputs

            content_tokens = tokenizer.convert_ids_to_tokens(window[0])

            candidates_indices = {}
            for phrase in candidates:
                matched_indices = get_phrase_indices(content_tokens, phrase, prefix)
                if len(matched_indices) == 0:
                    continue
                candidates_indices[phrase] = matched_indices
            candidates_indices = remove_repeated_sub_word(candidates_indices)
            all_indices = get_all_indices(candidates_indices,0)

            ###############################################################
            # ATTENTION MEASSUREMENT
            SAM_attentions = sum(attentions)/len(attentions)
            SAM_attentions = SAM_attentions.squeeze(0)
            SAM_attentions =  SAM_attentions.mean(0)
            del attentions
            ####################################################
            # GLOBAL SCORE
            att_scores = SAM_attentions.sum(0)
            ###############################################################
            for phrase in candidates_indices.keys():
                try:
                    phrase_indices = candidates_indices[phrase]
                    if len(phrase_indices) == 0:
                        continue
                except KeyError:
                    continue

                final_phrase_score = aggregate_phrase_scores(phrase_indices, att_scores)

                if len(phrase.split()) == 1:
                    final_phrase_score = final_phrase_score / len(phrase_indices)

                if phrase not in phrase_score_dict:
                    phrase_score_dict[phrase] = 0
                
                phrase_score_dict[phrase] += final_phrase_score
    
    sorted_scores_att_o_s = sorted(phrase_score_dict.items(), key=lambda item: item[1], reverse=True)
    stemmed_sorted_scores_att_o_s = [(" ".join(stemmer.stem(word) for word in phrase.split()), score) for
                                phrase, score in sorted_scores_att_o_s]

    set_stemmed_scores_list_att_o_s = []
    for phrase, score in stemmed_sorted_scores_att_o_s:
        if phrase not in set_stemmed_scores_list_att_o_s:
            set_stemmed_scores_list_att_o_s.append(phrase)

    pred_stemmed_phrases_att_o_s = set_stemmed_scores_list_att_o_s[:15]
    dataset_att_scores_overall.append(pred_stemmed_phrases_att_o_s)

att_o_s_top3_f1_5, att_o_s_top3_f1_10, att_o_s_top3_f1_15 = evaluate_dataset(dataset_att_scores_overall, dataset, "SAMRANK_G", dataset_name)

NON-PARAMETRIC SAMRANK PROPORTIONAL SCORE

In [8]:
model.to(device)
model.eval()

max_len = 1024

dataset_att_scores_overall = []

for data in tqdm(dataset):
    with torch.no_grad():
        input_text = data["title"] + ". " + data["abstract"] + " " + clean_data(data["fulltext"])
        tokenized_content = tokenizer(input_text, return_tensors='pt')

        candidates = get_candidates(pos_tagger, input_text)
        candidates = [phrase for phrase in candidates if phrase.split(' ')[0] not in stopwords]

        total_tokens_ids = tokenized_content['input_ids'].squeeze(0).tolist()
        total_tokens_ids = total_tokens_ids[1:]

        windows, attention_masks = get_same_len_segments(total_tokens_ids, max_len)

        phrase_score_dict = {}

        for i in range(len(windows)):
            window = windows[i]
            attention_mask = attention_masks[i]
            window = [128000] + window
            attention_mask = [1] + attention_mask
            window = torch.tensor([window])
            attention_mask = torch.tensor([attention_mask])

            outputs = model(window.to(device), attention_mask=attention_mask.to(device))
            attentions = outputs.attentions
            del outputs

            content_tokens = tokenizer.convert_ids_to_tokens(window[0])

            candidates_indices = {}
            for phrase in candidates:
                matched_indices = get_phrase_indices(content_tokens, phrase, prefix)
                if len(matched_indices) == 0:
                    continue
                candidates_indices[phrase] = matched_indices
            candidates_indices = remove_repeated_sub_word(candidates_indices)
            all_indices = get_all_indices(candidates_indices,0)

            ###############################################################
            # ATTENTION MEASSUREMENT
            SAM_attentions = sum(attentions)/len(attentions)
            SAM_attentions = SAM_attentions.squeeze(0)
            SAM_attentions =  SAM_attentions.mean(0)
            del attentions
            ####################################################
            # PROPORTIONAL SCORE
            SAM_attentions_sum = SAM_attentions.sum(0, keepdim=True)
            SAM_attentions_sum += 1e-10
            SAM_attentions_norm = SAM_attentions / SAM_attentions_sum
            SAM_attentions_norm[:,0] = 0
            p_att_scores = SAM_attentions_norm.sum(1)
            ###############################################################
            for phrase in candidates_indices.keys():
                try:
                    phrase_indices = candidates_indices[phrase]
                    if len(phrase_indices) == 0:
                        continue
                except KeyError:
                    continue

                final_phrase_score = aggregate_phrase_scores(phrase_indices, p_att_scores)

                if len(phrase.split()) == 1:
                    final_phrase_score = final_phrase_score / len(phrase_indices)

                if phrase not in phrase_score_dict:
                    phrase_score_dict[phrase] = 0
                
                phrase_score_dict[phrase] += final_phrase_score
    
    sorted_scores_att_o_s = sorted(phrase_score_dict.items(), key=lambda item: item[1], reverse=True)
    stemmed_sorted_scores_att_o_s = [(" ".join(stemmer.stem(word) for word in phrase.split()), score) for
                                phrase, score in sorted_scores_att_o_s]

    set_stemmed_scores_list_att_o_s = []
    for phrase, score in stemmed_sorted_scores_att_o_s:
        if phrase not in set_stemmed_scores_list_att_o_s:
            set_stemmed_scores_list_att_o_s.append(phrase)

    pred_stemmed_phrases_att_o_s = set_stemmed_scores_list_att_o_s[:15]
    dataset_att_scores_overall.append(pred_stemmed_phrases_att_o_s)

att_o_s_top3_f1_5, att_o_s_top3_f1_10, att_o_s_top3_f1_15 = evaluate_dataset(dataset_att_scores_overall, dataset, "SAMRANK_P", dataset_name)

100%|██████████| 100/100 [19:48<00:00, 11.89s/it]

########################
Metrics
@5
F1:0.15740101928694833
P:0.306
R:0.1070442085698397
@10
F1:0.1907137869354619
P:0.23600000000000002
R:0.1629700227763607
@15
F1:0.19336612613349075
P:0.19199999999999995
R:0.19913201291900098
#########################


NON-PARAMETRIC SAMRANK

In [9]:
model.to(device)
model.eval()

max_len = 1024

dataset_att_scores_overall = []

for data in tqdm(dataset):
    with torch.no_grad():
        input_text = data["title"] + ". " + data["abstract"] + " " + clean_data(data["fulltext"])
        tokenized_content = tokenizer(input_text, return_tensors='pt')

        candidates = get_candidates(pos_tagger, input_text)
        candidates = [phrase for phrase in candidates if phrase.split(' ')[0] not in stopwords]

        total_tokens_ids = tokenized_content['input_ids'].squeeze(0).tolist()
        total_tokens_ids = total_tokens_ids[1:]

        windows, attention_masks = get_same_len_segments(total_tokens_ids, max_len)

        phrase_score_dict = {}

        for i in range(len(windows)):
            window = windows[i]
            attention_mask = attention_masks[i]
            window = [128000] + window
            attention_mask = [1] + attention_mask
            window = torch.tensor([window])
            attention_mask = torch.tensor([attention_mask])

            outputs = model(window.to(device), attention_mask=attention_mask.to(device))
            attentions = outputs.attentions
            del outputs

            content_tokens = tokenizer.convert_ids_to_tokens(window[0])

            candidates_indices = {}
            for phrase in candidates:
                matched_indices = get_phrase_indices(content_tokens, phrase, prefix)
                if len(matched_indices) == 0:
                    continue
                candidates_indices[phrase] = matched_indices
            candidates_indices = remove_repeated_sub_word(candidates_indices)
            all_indices = get_all_indices(candidates_indices,0)

            ###############################################################
            # ATTENTION MEASSUREMENT
            SAM_attentions = sum(attentions)/len(attentions)
            SAM_attentions = SAM_attentions.squeeze(0)
            SAM_attentions =  SAM_attentions.mean(0)
            del attentions
            ####################################################
            # GLOBAL SCORE
            att_scores = SAM_attentions.sum(0)
            # PROPORTIONAL SCORE
            SAM_attentions_sum = SAM_attentions.sum(0, keepdim=True)
            SAM_attentions_sum += 1e-10
            SAM_attentions_norm = SAM_attentions / SAM_attentions_sum
            SAM_attentions_norm[:,0] = 0
            p_att_scores = SAM_attentions_norm.sum(1)
            # FINAL SCORE = GLOBAL SCORE + PROPORTIONAL SCORE
            att_scores = att_scores + p_att_scores
            ###############################################################
            for phrase in candidates_indices.keys():
                try:
                    phrase_indices = candidates_indices[phrase]
                    if len(phrase_indices) == 0:
                        continue
                except KeyError:
                    continue

                final_phrase_score = aggregate_phrase_scores(phrase_indices, att_scores)

                if len(phrase.split()) == 1:
                    final_phrase_score = final_phrase_score / len(phrase_indices)

                if phrase not in phrase_score_dict:
                    phrase_score_dict[phrase] = 0
                
                phrase_score_dict[phrase] += final_phrase_score
    
    sorted_scores_att_o_s = sorted(phrase_score_dict.items(), key=lambda item: item[1], reverse=True)
    stemmed_sorted_scores_att_o_s = [(" ".join(stemmer.stem(word) for word in phrase.split()), score) for
                                phrase, score in sorted_scores_att_o_s]

    set_stemmed_scores_list_att_o_s = []
    for phrase, score in stemmed_sorted_scores_att_o_s:
        if phrase not in set_stemmed_scores_list_att_o_s:
            set_stemmed_scores_list_att_o_s.append(phrase)

    pred_stemmed_phrases_att_o_s = set_stemmed_scores_list_att_o_s[:15]
    dataset_att_scores_overall.append(pred_stemmed_phrases_att_o_s)

att_o_s_top3_f1_5, att_o_s_top3_f1_10, att_o_s_top3_f1_15 = evaluate_dataset(dataset_att_scores_overall, dataset, "SAMRANK_GP", dataset_name)

100%|██████████| 100/100 [19:55<00:00, 11.96s/it]


########################
Metrics
@5
F1:0.16359165078021143
P:0.32
R:0.1110523984793454
@10
F1:0.20131771838340032
P:0.24900000000000003
R:0.17217069504011345
@15
F1:0.2087377272906505
P:0.20733333333333326
R:0.21479329430411798
#########################


<h2>DATASET: KRAPIVIN</h2>

In [10]:
dataset_name = "krapivin_test"
dataset = read_jsonl("KEYWORD_DATA/{}.json".format(dataset_name))

NON-PARAMETRIC SAMRANK GLOBAL SCORE

In [ ]:
model.to(device)
model.eval()

max_len = 1024

dataset_att_scores_overall = []

for data in tqdm(dataset):
    with torch.no_grad():
        input_text = data["title"] + " " + data["abstract"] + " " + clean_data(data["fulltext"])
        tokenized_content = tokenizer(input_text, return_tensors='pt')

        try:
            candidates = get_candidates(pos_tagger, input_text)
            candidates = [phrase for phrase in candidates if phrase.split(' ')[0] not in stopwords]
        except:
            input_len = len(input_text)
            candidates = get_candidates(pos_tagger, input_text[:int(input_len/2)])
            candidates = [phrase for phrase in candidates if phrase.split(' ')[0] not in stopwords]
            candidates_2 = get_candidates(pos_tagger, input_text[int(input_len/2):])
            candidates_2 = [phrase for phrase in candidates_2 if phrase.split(' ')[0] not in stopwords]
            candidates.extend(candidates_2)

        total_tokens_ids = tokenized_content['input_ids'].squeeze(0).tolist()
        total_tokens_ids = total_tokens_ids[1:]

        windows, attention_masks = get_same_len_segments(total_tokens_ids, max_len)

        phrase_score_dict = {}

        for i in range(len(windows)):
            window = windows[i]
            attention_mask = attention_masks[i]
            window = [128000] + window
            attention_mask = [1] + attention_mask
            window = torch.tensor([window])
            attention_mask = torch.tensor([attention_mask])

            outputs = model(window.to(device), attention_mask=attention_mask.to(device))
            attentions = outputs.attentions
            del outputs

            content_tokens = tokenizer.convert_ids_to_tokens(window[0])

            candidates_indices = {}
            for phrase in candidates:
                matched_indices = get_phrase_indices(content_tokens, phrase, prefix)
                if len(matched_indices) == 0:
                    continue
                candidates_indices[phrase] = matched_indices
            candidates_indices = remove_repeated_sub_word(candidates_indices)
            all_indices = get_all_indices(candidates_indices,0)

            ###############################################################
            # ATTENTION MEASSUREMENT
            SAM_attentions = sum(attentions)/len(attentions)
            SAM_attentions = SAM_attentions.squeeze(0)
            SAM_attentions =  SAM_attentions.mean(0)
            del attentions
            ####################################################
            # GLOBAL SCORE
            att_scores = SAM_attentions.sum(0)
            ###############################################################
            for phrase in candidates_indices.keys():
                try:
                    phrase_indices = candidates_indices[phrase]
                    if len(phrase_indices) == 0:
                        continue
                except KeyError:
                    continue

                final_phrase_score = aggregate_phrase_scores(phrase_indices, att_scores)

                if len(phrase.split()) == 1:
                    final_phrase_score = final_phrase_score / len(phrase_indices)

                if phrase not in phrase_score_dict:
                    phrase_score_dict[phrase] = 0
                
                phrase_score_dict[phrase] += final_phrase_score
    
    sorted_scores_att_o_s = sorted(phrase_score_dict.items(), key=lambda item: item[1], reverse=True)
    stemmed_sorted_scores_att_o_s = [(" ".join(stemmer.stem(word) for word in phrase.split()), score) for
                                phrase, score in sorted_scores_att_o_s]

    set_stemmed_scores_list_att_o_s = []
    for phrase, score in stemmed_sorted_scores_att_o_s:
        if phrase not in set_stemmed_scores_list_att_o_s:
            set_stemmed_scores_list_att_o_s.append(phrase)

    pred_stemmed_phrases_att_o_s = set_stemmed_scores_list_att_o_s[:15]
    dataset_att_scores_overall.append(pred_stemmed_phrases_att_o_s)

att_o_s_top3_f1_5, att_o_s_top3_f1_10, att_o_s_top3_f1_15 = evaluate_dataset(dataset_att_scores_overall, dataset, "SAMRANK_G", dataset_name)

NON-PARAMETRIC SAMRANK PROPORTIONAL SCORE

In [11]:
model.to(device)
model.eval()

max_len = 1024

dataset_att_scores_overall = []

for data in tqdm(dataset):
    with torch.no_grad():
        input_text = data["title"] + " " + data["abstract"] + " " + clean_data(data["fulltext"])
        tokenized_content = tokenizer(input_text, return_tensors='pt')

        try:
            candidates = get_candidates(pos_tagger, input_text)
            candidates = [phrase for phrase in candidates if phrase.split(' ')[0] not in stopwords]
        except:
            input_len = len(input_text)
            candidates = get_candidates(pos_tagger, input_text[:int(input_len/2)])
            candidates = [phrase for phrase in candidates if phrase.split(' ')[0] not in stopwords]
            candidates_2 = get_candidates(pos_tagger, input_text[int(input_len/2):])
            candidates_2 = [phrase for phrase in candidates_2 if phrase.split(' ')[0] not in stopwords]
            candidates.extend(candidates_2)

        total_tokens_ids = tokenized_content['input_ids'].squeeze(0).tolist()
        total_tokens_ids = total_tokens_ids[1:]

        windows, attention_masks = get_same_len_segments(total_tokens_ids, max_len)

        phrase_score_dict = {}

        for i in range(len(windows)):
            window = windows[i]
            attention_mask = attention_masks[i]
            window = [128000] + window
            attention_mask = [1] + attention_mask
            window = torch.tensor([window])
            attention_mask = torch.tensor([attention_mask])

            outputs = model(window.to(device), attention_mask=attention_mask.to(device))
            attentions = outputs.attentions
            del outputs

            content_tokens = tokenizer.convert_ids_to_tokens(window[0])

            candidates_indices = {}
            for phrase in candidates:
                matched_indices = get_phrase_indices(content_tokens, phrase, prefix)
                if len(matched_indices) == 0:
                    continue
                candidates_indices[phrase] = matched_indices
            candidates_indices = remove_repeated_sub_word(candidates_indices)
            all_indices = get_all_indices(candidates_indices,0)

            ###############################################################
            # ATTENTION MEASSUREMENT
            SAM_attentions = sum(attentions)/len(attentions)
            SAM_attentions = SAM_attentions.squeeze(0)
            SAM_attentions =  SAM_attentions.mean(0)
            del attentions
            ####################################################
            # PROPORTIONAL SCORE
            SAM_attentions_sum = SAM_attentions.sum(0, keepdim=True)
            SAM_attentions_sum += 1e-10
            SAM_attentions_norm = SAM_attentions / SAM_attentions_sum
            SAM_attentions_norm[:,0] = 0
            p_att_scores = SAM_attentions_norm.sum(1)
            ###############################################################
            for phrase in candidates_indices.keys():
                try:
                    phrase_indices = candidates_indices[phrase]
                    if len(phrase_indices) == 0:
                        continue
                except KeyError:
                    continue

                final_phrase_score = aggregate_phrase_scores(phrase_indices, p_att_scores)

                if len(phrase.split()) == 1:
                    final_phrase_score = final_phrase_score / len(phrase_indices)

                if phrase not in phrase_score_dict:
                    phrase_score_dict[phrase] = 0
                
                phrase_score_dict[phrase] += final_phrase_score
    
    sorted_scores_att_o_s = sorted(phrase_score_dict.items(), key=lambda item: item[1], reverse=True)
    stemmed_sorted_scores_att_o_s = [(" ".join(stemmer.stem(word) for word in phrase.split()), score) for
                                phrase, score in sorted_scores_att_o_s]

    set_stemmed_scores_list_att_o_s = []
    for phrase, score in stemmed_sorted_scores_att_o_s:
        if phrase not in set_stemmed_scores_list_att_o_s:
            set_stemmed_scores_list_att_o_s.append(phrase)

    pred_stemmed_phrases_att_o_s = set_stemmed_scores_list_att_o_s[:15]
    dataset_att_scores_overall.append(pred_stemmed_phrases_att_o_s)

att_o_s_top3_f1_5, att_o_s_top3_f1_10, att_o_s_top3_f1_15 = evaluate_dataset(dataset_att_scores_overall, dataset, "SAMRANK_P", dataset_name)

100%|██████████| 460/460 [1:31:56<00:00, 11.99s/it]

########################
Metrics
@5
F1:0.14839623885901548
P:0.15434782608695652
R:0.15780140787518968
@10
F1:0.14445672396882767
P:0.11413043478260869
R:0.2286932833827604
@15
F1:0.13273655412600835
P:0.0927536231884058
R:0.27771800541705643
#########################


NON-PARAMETRIC SAMRANK

In [12]:
model.to(device)
model.eval()

max_len = 1024

dataset_att_scores_overall = []

for data in tqdm(dataset):
    with torch.no_grad():
        input_text = data["title"] + " " + data["abstract"] + " " + clean_data(data["fulltext"])
        tokenized_content = tokenizer(input_text, return_tensors='pt')

        try:
            candidates = get_candidates(pos_tagger, input_text)
            candidates = [phrase for phrase in candidates if phrase.split(' ')[0] not in stopwords]
        except:
            input_len = len(input_text)
            candidates = get_candidates(pos_tagger, input_text[:int(input_len/2)])
            candidates = [phrase for phrase in candidates if phrase.split(' ')[0] not in stopwords]
            candidates_2 = get_candidates(pos_tagger, input_text[int(input_len/2):])
            candidates_2 = [phrase for phrase in candidates_2 if phrase.split(' ')[0] not in stopwords]
            candidates.extend(candidates_2)

        total_tokens_ids = tokenized_content['input_ids'].squeeze(0).tolist()
        total_tokens_ids = total_tokens_ids[1:]

        windows, attention_masks = get_same_len_segments(total_tokens_ids, max_len)

        phrase_score_dict = {}

        for i in range(len(windows)):
            window = windows[i]
            attention_mask = attention_masks[i]
            window = [128000] + window
            attention_mask = [1] + attention_mask
            window = torch.tensor([window])
            attention_mask = torch.tensor([attention_mask])

            outputs = model(window.to(device), attention_mask=attention_mask.to(device))
            attentions = outputs.attentions
            del outputs

            content_tokens = tokenizer.convert_ids_to_tokens(window[0])

            candidates_indices = {}
            for phrase in candidates:
                matched_indices = get_phrase_indices(content_tokens, phrase, prefix)
                if len(matched_indices) == 0:
                    continue
                candidates_indices[phrase] = matched_indices
            candidates_indices = remove_repeated_sub_word(candidates_indices)
            all_indices = get_all_indices(candidates_indices,0)

            ###############################################################
            # ATTENTION MEASSUREMENT
            SAM_attentions = sum(attentions)/len(attentions)
            SAM_attentions = SAM_attentions.squeeze(0)
            SAM_attentions =  SAM_attentions.mean(0)
            del attentions
            ####################################################
            # GLOBAL SCORE
            att_scores = SAM_attentions.sum(0)
            # PROPORTIONAL SCORE
            SAM_attentions_sum = SAM_attentions.sum(0, keepdim=True)
            SAM_attentions_sum += 1e-10
            SAM_attentions_norm = SAM_attentions / SAM_attentions_sum
            SAM_attentions_norm[:,0] = 0
            p_att_scores = SAM_attentions_norm.sum(1)
            # FINAL SCORE = GLOBAL SCORE + PROPORTIONAL SCORE
            att_scores = att_scores + p_att_scores
            ###############################################################
            for phrase in candidates_indices.keys():
                try:
                    phrase_indices = candidates_indices[phrase]
                    if len(phrase_indices) == 0:
                        continue
                except KeyError:
                    continue

                final_phrase_score = aggregate_phrase_scores(phrase_indices, att_scores)

                if len(phrase.split()) == 1:
                    final_phrase_score = final_phrase_score / len(phrase_indices)

                if phrase not in phrase_score_dict:
                    phrase_score_dict[phrase] = 0
                
                phrase_score_dict[phrase] += final_phrase_score
    
    sorted_scores_att_o_s = sorted(phrase_score_dict.items(), key=lambda item: item[1], reverse=True)
    stemmed_sorted_scores_att_o_s = [(" ".join(stemmer.stem(word) for word in phrase.split()), score) for
                                phrase, score in sorted_scores_att_o_s]

    set_stemmed_scores_list_att_o_s = []
    for phrase, score in stemmed_sorted_scores_att_o_s:
        if phrase not in set_stemmed_scores_list_att_o_s:
            set_stemmed_scores_list_att_o_s.append(phrase)

    pred_stemmed_phrases_att_o_s = set_stemmed_scores_list_att_o_s[:15]
    dataset_att_scores_overall.append(pred_stemmed_phrases_att_o_s)

att_o_s_top3_f1_5, att_o_s_top3_f1_10, att_o_s_top3_f1_15 = evaluate_dataset(dataset_att_scores_overall, dataset, "SAMRANK_GP", dataset_name)

100%|██████████| 460/460 [1:31:29<00:00, 11.93s/it]

########################
Metrics
@5
F1:0.16056860100973802
P:0.16695652173913042
R:0.1716572169212161
@10
F1:0.15455253635502086
P:0.12260869565217393
R:0.24353579292635694
@15
F1:0.14290633866509722
P:0.10014492753623187
R:0.29752880553922073
#########################
